In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="-1"  # To disable GPU
import torch
# Check if CUDA (GPU) is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Device:", device)


Device: cpu


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score


# Load the dataset
df = pd.read_csv("spanish.csv", encoding='latin1')


# Trim the dataset to match the number of samples in the embeddings
df = df.head(33646)

# Assuming the embeddings are already saved in a numpy file
embeddings_list = np.load("Embeddings/MT5/SPANISH/spanish_mt5_transliterated_train_embeds.npy")

# Assuming the labels are in the 'label' column of the dataframe
labels = df['class']

# Encode the labels
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(embeddings_list, labels, test_size=0.2, random_state=42)

# Define a simple deep neural network model
model = Sequential([
    Flatten(input_shape=(X_train.shape[1:])),  # Flatten the 3D embeddings to 1D
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1)

# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test Accuracy:", test_accuracy)


/home/abcd/anaconda3/envs/abcd/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2024-03-27 23:54:38.143136: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-27 23:54:38.277939: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-27 23:54:38.306075: E tensorflow/stream_executor/cuda/cuda_blas.cc:

Epoch 1/10
757/757 [==============================] - 55s 71ms/step - loss: 0.0053 - accuracy: 0.9995 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/10
757/757 [==============================] - 53s 70ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/10
757/757 [==============================] - 53s 70ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/10
757/757 [==============================] - 53s 70ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 5/10
757/757 [==============================] - ETA: 0s - loss: 0.0000e+00 - accuracy: 1.0000

KeyboardInterrupt: 

In [8]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

y_pred = (model.predict(X_test) > 0.5).astype("int32")  # Convert probabilities to binary predictions
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred,zero_division=0,average='weighted')
recall = recall_score(y_test, y_pred,zero_division=0,average='weighted')
f1 = f1_score(y_test, y_pred,zero_division=0,average='weighted')

print("Test Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

211/211 [==============================] - 4s 20ms/step
Test Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1 Score: 1.0
